# Predict Bankruptcy

### 1) Load required modules

In [1]:
import pandas                   as     pd
import numpy                    as     np
import seaborn                  as     sns
import matplotlib.pyplot        as     plt
import statsmodels.api          as     sm
from   sklearn.preprocessing    import OrdinalEncoder
from   sklearn                  import metrics
from   sklearn                  import tree
import sys, os
import os
os.chdir(r'D:\DrPKV\20220618')

In [2]:
import time
from    datetime   import datetime
from    datetime   import timedelta

In [3]:
### Modules for imputing
from sklearn.experimental  import   enable_iterative_imputer
from sklearn.impute        import   IterativeImputer
from sklearn.linear_model   import   LinearRegression


In [4]:
### Explore the algorithm wrapped by RFE
###
from numpy                         import mean
from numpy                         import std
from sklearn.datasets              import make_classification
from sklearn.model_selection       import cross_val_score
from sklearn.model_selection       import RepeatedStratifiedKFold
from sklearn.feature_selection     import RFE
from sklearn.feature_selection     import RFECV
###
from sklearn.linear_model          import LogisticRegression
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.svm                   import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.pipeline              import Pipeline
from matplotlib                    import pyplot as plt
###

In [5]:
pd.set_option('display.float_format', lambda x: '%.5f' % x) # To get rid of E notation

### Define required functions

####  Function to detect zero & null values and report column-wise count & percentage of zero & missing values

In [6]:
"""
Function name : missing_zero_values_table
Arguments:
Input: Dataset name
Output : Report on column-wise count & percentage of zero & missing values

"""
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero & Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero & Missing Values'] = 100 * mz_table['Total Zero & Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

In [7]:
### ------- -------------- -------------- -----------
### get a list of models to evaluate
### ------- -------------- -------------- -----------
def get_models():
    models = dict()
    # 1) Logistic Regression
    rfe            = RFE(estimator=LogisticRegression(), n_features_to_select = 10)
    model          = DecisionTreeClassifier()
    models['lr']   = Pipeline(steps=[('s',rfe),('m',model)])
    # 2) CART / Decision Tree
    rfe            = RFE(estimator=DecisionTreeClassifier(), n_features_to_select = 10)
    model          = DecisionTreeClassifier()
    models['cart'] = Pipeline(steps=[('s',rfe),('m',model)])
    # 3) Random Forest
    rfe           = RFE(estimator=RandomForestClassifier(), n_features_to_select = 10)
    model         = RandomForestClassifier()
    models['rf']  = Pipeline(steps=[('s',rfe),('m',model)])
    # 4) SVC
    rfe           = RFE(estimator=SVC(), n_features_to_select = 10)
    model         = SVC()
    models['SVM'] = Pipeline(steps=[('s',rfe),('m',model)])
    # 5) LDA
    rfe           = RFE(estimator=LinearDiscriminantAnalysis(), n_features_to_select = 10)
    model         = LinearDiscriminantAnalysis()
    models['LDA'] = Pipeline(steps=[('s',rfe),('m',model)])    
    # 6) KNN
    rfe           = RFE(estimator=KNeighborsClassifier(), n_features_to_select = 10)
    model         = KNeighborsClassifier()
    models['LDA'] = Pipeline(steps=[('s',rfe),('m',model)])        
    
    return models
 

In [8]:
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

### 2) Read dataset from an Excel file

In [9]:
file_name = './Data/BankruptcyData.xlsx'

In [10]:
df     =   pd.read_excel(file_name, sheet_name = 'Full data')

### 3) Data Pre-processing

#### 3.1)  To know the data type of all the variables

In [11]:
df.info() # to know the structure of dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 34 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Row                                       9000 non-null   int64  
 1   Year                                      9000 non-null   int64  
 2   Company_name                              9000 non-null   object 
 3   Year Encoded                              9000 non-null   int64  
 4    Dummy Coded :Healthy=1; NPA= 0           9000 non-null   int64  
 5   Asset_turnover                            8299 non-null   float64
 6   Receivable_turnover(new)                  7390 non-null   float64
 7   Inventory_turnover                        5702 non-null   float64
 8   Cash_ratio                                8278 non-null   float64
 9   Quick_ratio                               8278 non-null   float64
 10  Current_ratio                       

### Observations

**We have 9000 observations and 34 columns in this dataset**

**We find from the data dictionary, the column names and description.**

|S.No | Column name | Description |
|--- | :------------------- |:-----------------------------------| 
| 1 | Row | Company code |
| 2 | Year	| year |
| 3 | Company_name | Company Name |
| 4 | Year Encoded	| year; 0 means latest year |
| 5 | Dummy Coded :Healthy=1; NPA= 0 | NPA = 0 and 1= Healthy, Our Target variable |
| 6 | Asset_turnover | Total income/ Total assets |
| 7 | Receivable_turnover(new) | Net sales/Total assets |
| 8 | Inventory_turnover | 	COGS/ Total inventories |
| 9 | Cash_ratio | Cash and cash balance/ Total Current liabilities | 
| 10 | Quick_ratio | Cash and Cash Equivalents + Receivables + Marketable securities/T Total Current liabilities | 
| 11 | Current_ratio | Current asset/current liabilities | 
| 12 | ROA(new) | Net income/Total assets | 
| 13 | ROE(new) | Net income/Shareholder's equity | 
| 14 | ROS(new) | Net income/Total sales | 
| 15 | ROI(new) | Net income/Total investment | 
| 16 | debt_asset | Total debt/Total asset | 
| 17 | debt_equity | Total debt/ Total equity | 
| 18 | debt_income | Total debt / EBIT | 
| 19 | Interest_coverage | EBITDA/Interest | 
| 20 | Asset_coverage | Total asset - (CA-CL) / (Total debt) | 
| 21 | EBIT_Sales | EBIT/Total sales | 
| 22 | Sales_CE | Sales/Total capital employed | 
| 23 | ROCE_CE | (EBIT/Sales) * (Sales/CE) | 
| 24 | Changeinsales_Industry | Sales (current year)- Sales (Previous year)/ Sales (current year) | 
| 25 | Grossvaluedadded | Grossvaluedadded/Total grossvaluedadded | 
| 26 | Ln_GVA | Ln (Gross value added) | 
| 27 | Operating Cash Flow/Total Sales | Operating cash flow/Total sales | 
| 28 | Operating Cash Flow/Total Debt | Operating cash flow/Total debt | 
| 29 | Operating Cash Flow/Shareholder's Equity | Operating cash flow /Total equity | 
| 30 | Fixed Asset Turnover Ratio | Total income/ Fixed asset | 
| 31 | YOY Sales Growth Rate | Y-O-Y Sales Growth rate  | 
| 32 | YOY EBIT Growth Rate | Y-O-Y EBIT Growth rate  | 
| 33 | Total shareholders' funds | Total shareholder's equity | 
| 34 | Shareholderquity_code | Dummy variable 	0 = healthy firm +ve equity,	1 = healthy firm -ve equity,2 = banktrupt firm +ve, equity,	3 = banktrupt firm -ve equity |

We observe that the following columns are not required
* 1) Row representing the company code
* 2) Company_name
- We are more interested in knowing the characteristics of the company going bankrupt rather than the company name
* 3) Year
We already have the column Year Encoded representing the year. So we remove one of them and hence Year is removed

In [12]:
unwanted_cols   =   ['Row', 'Year', 'Company_name']
df.drop(unwanted_cols, axis = 1, inplace = True)

### Rename the target variable, "  Dummy Coded :Healthy=1; NPA= 0" as "Target"

In [13]:
colnames =  df.columns
colnames

Index(['Year Encoded', ' Dummy Coded :Healthy=1; NPA= 0', 'Asset_turnover',
       'Receivable_turnover(new)', 'Inventory_turnover', 'Cash_ratio',
       'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)', 'ROS(new)',
       'ROI(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       'Operating Cash Flow/Shareholder's Equity',
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', 'Total shareholders' funds',
       'Shareholderquity_code'],
      dtype='object')

In [14]:
new_colnames = ["Year Encoded", "Target", "Asset_turnover", "Receivable_turnover(new)", "Inventory_turnover", "Cash_ratio",\
       "Quick_ratio", "Current_ratio", "ROA(new)", "ROE(new)", "ROS(new)","ROI(new)", "debt_asset", "debt_equity",\
       "debt_income","Interest_coverage", "Asset_coverage", "EBIT_Sales", "Sales_CE", "ROCE_CE", "Changeinsales_Industry",\
       "Grossvaluedadded", "Ln_GVA", "Operating Cash Flow/Total Sales", "Operating Cash Flow/Total Debt",\
       "Operating Cash Flow/Shareholder's Equity","Fixed Asset Turnover Ratio", "YOY Sales Growth Rate","YOY EBIT Growth Rate",\
       "Total shareholders' funds", "Shareholderquity_code"]

In [15]:
df.columns = new_colnames

#### 3.2)  To know about missing values - Column-wise  count & Percentage of missing values 

In [16]:
missing_zero_values_table(df)

Your selected dataframe has 31 columns and 9000 Rows.
There are 29 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero & Missing Values,% Total Zero & Missing Values,Data Type
ROI(new),25,4132,45.90000,4157,46.20000,float64
Shareholderquity_code,3305,3790,42.10000,7095,78.80000,float64
Inventory_turnover,4364,3298,36.60000,7662,85.10000,float64
Receivable_turnover(new),401,1610,17.90000,2011,22.30000,float64
Interest_coverage,218,1569,17.40000,1787,19.90000,float64
Operating Cash Flow/Total Debt,15,1306,14.50000,1321,14.70000,float64
Asset_coverage,0,1306,14.50000,1306,14.50000,float64
ROCE_CE,70,1258,14.00000,1328,14.80000,float64
Fixed Asset Turnover Ratio,476,1007,11.20000,1483,16.50000,float64
Cash_ratio,21,722,8.00000,743,8.30000,float64


### Observations

* We observe that there are 29 columns that have missing values. We do not want to incur data loss by removing rows with missing values. Let us impute them.

* Detecting and handling missing values in the correct way is important, as they can impact the results of the analysis. It cannot be imputed with general ways of using mean, mode, or median which ignores the inherent relationship among data and also it can pollute the data. 

* We observe that on a few occasions, data is missing in a dataset and is related to the other features and hence they can be predicted using other feature values.  Imputing by prediction of missing values is superior to other techniques since the inherent relationship among data is not ignored.

* We are imputing missing numerical values using the IterativeImputer class in sklearn. 


Ref: https://www.numpyninja.com/post/mice-and-knn-missing-value-imputations-through-python



#### 3.3)  Imputation of missing values through Multiple Imputation by Chained Equation

In [17]:
lreg         =  LinearRegression()
imp          =  IterativeImputer(estimator= lreg, missing_values = np.nan, max_iter = 10, verbose = 2,\
                                 imputation_order= 'roman',random_state = 0)
X            =  imp.fit_transform(df)

[IterativeImputer] Completing matrix with shape (9000, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.22
[IterativeImputer] Change: 12802672118.836647, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.42
[IterativeImputer] Change: 2504019127.796545, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.64
[IterativeImputer] Change: 2375148955.5433493, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.85
[IterativeImputer] Change: 1492808165.906601, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 5/10, elapsed time 1.05
[IterativeImputer] Change: 1545504463.0583532, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 6/10, elapsed time 1.29
[IterativeImputer] Change: 1344858451.117177, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 7/10, elapsed time 1.48
[Iterat

C:\Users\User\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [18]:
X_df    =    pd.DataFrame(X, columns = df.columns)
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year Encoded                              9000 non-null   float64
 1   Target                                    9000 non-null   float64
 2   Asset_turnover                            9000 non-null   float64
 3   Receivable_turnover(new)                  9000 non-null   float64
 4   Inventory_turnover                        9000 non-null   float64
 5   Cash_ratio                                9000 non-null   float64
 6   Quick_ratio                               9000 non-null   float64
 7   Current_ratio                             9000 non-null   float64
 8   ROA(new)                                  9000 non-null   float64
 9   ROE(new)                                  9000 non-null   float64
 10  ROS(new)                            

### Observations

All the variables are of float data type. We shall convert the following variables to integer data type:
* 1) Year Encoded
* 2) Target

In [19]:
X_df['Year Encoded']                         =  X_df['Year Encoded'].astype(int)
X_df['Target']                               =  X_df['Target'].astype(int)

In [20]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year Encoded                              9000 non-null   int32  
 1   Target                                    9000 non-null   int32  
 2   Asset_turnover                            9000 non-null   float64
 3   Receivable_turnover(new)                  9000 non-null   float64
 4   Inventory_turnover                        9000 non-null   float64
 5   Cash_ratio                                9000 non-null   float64
 6   Quick_ratio                               9000 non-null   float64
 7   Current_ratio                             9000 non-null   float64
 8   ROA(new)                                  9000 non-null   float64
 9   ROE(new)                                  9000 non-null   float64
 10  ROS(new)                            

### Change the order of data frame

In [21]:
new_cols  = ["Target", "Year Encoded", "Asset_turnover", "Receivable_turnover(new)", "Inventory_turnover", "Cash_ratio",\
       "Quick_ratio", "Current_ratio", "ROA(new)", "ROE(new)", "ROS(new)","ROI(new)", "debt_asset", "debt_equity",\
       "debt_income","Interest_coverage", "Asset_coverage", "EBIT_Sales", "Sales_CE", "ROCE_CE", "Changeinsales_Industry",\
       "Grossvaluedadded", "Ln_GVA", "Operating Cash Flow/Total Sales", "Operating Cash Flow/Total Debt",\
       "Operating Cash Flow/Shareholder's Equity","Fixed Asset Turnover Ratio", "YOY Sales Growth Rate","YOY EBIT Growth Rate",\
       "Total shareholders' funds", "Shareholderquity_code"] 
df_imputed = X_df[new_cols]

In [22]:
df_imputed.head().T

,0,1,2,3,4
Target,1.00000,1.00000,1.00000,1.00000,1.00000
Year Encoded,-8.00000,-7.00000,-6.00000,-5.00000,-4.00000
Asset_turnover,0.00000,0.75666,1.07063,1.18632,1.42893
Receivable_turnover(new),0.00000,3.46077,3.95393,4.54417,6.10799
Inventory_turnover,0.00000,0.00000,0.00000,0.00000,0.00000
Cash_ratio,2.01764,1.55843,0.85337,0.96842,1.21196
Quick_ratio,1.55202,1.08407,0.38471,0.37537,0.61504
Current_ratio,3.73717,3.03393,2.56104,2.98033,3.18905
ROA(new),0.31478,0.29401,0.38998,0.38976,0.43305
ROE(new),0.39534,0.38944,0.52113,0.49546,0.55367


#### 3.4)  Feature Engineering

*Recursive Feature Elimination, or RFE for short*, is a popular feature selection algorithm in a dataset that are more or most relevant in predicting the target variable.

**RFE** applies a backward selection process to find the best combination of features. This is done as follows:
* 1) Builds a model based on all features and calculates the importance of each feature in the model.
* 2) It ranks the features and removes the feature(s) with the least importance iteratively based on model evaluation
metrics such as accuracy ratio.

Ref. https://towardsdatascience.com/effective-feature-selection-recursive-feature-elimination-using-r-148ff998e4f7

In [23]:
# define dataset
y             =    df_imputed['Target']
X             =    df_imputed.drop(['Target'], axis = 1)                       
print('X dimension {}'. format(X.shape))
print('y dimension {}'. format(y.shape))

X dimension (9000, 30)
y dimension (9000,)


In [24]:
collist =  X.columns

In [25]:
model_L         =    list()
df_imp          =    pd.DataFrame()

In [26]:
# define RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select = 10)
# fit RFE
rfe.fit(X, y)
# summarize all features

sno_L           =    list()
support_text    =    list()
rank_list       =    list()
each            =    'Decision Tree'
print("Proecessing %s" % each)
for i in range(X.shape[1]):
    model_L.append(each)
    sno_L.append(i)
    support_text.append(rfe.support_[i])
    rank_list.append(rfe.ranking_[i])
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    
df_imp_ind1   =    pd.DataFrame({'Sno' : sno_L, 'Model' : model_L, 'Selected?' : support_text, 'Rank' : rank_list, 'Columns' : collist })


Proecessing Decision Tree
Column: 0, Selected False, Rank: 21.000
Column: 1, Selected False, Rank: 20.000
Column: 2, Selected False, Rank: 19.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected False, Rank: 9.000
Column: 5, Selected False, Rank: 7.000
Column: 6, Selected False, Rank: 6.000
Column: 7, Selected False, Rank: 5.000
Column: 8, Selected False, Rank: 4.000
Column: 9, Selected False, Rank: 3.000
Column: 10, Selected False, Rank: 2.000
Column: 11, Selected False, Rank: 11.000
Column: 12, Selected False, Rank: 13.000
Column: 13, Selected False, Rank: 15.000
Column: 14, Selected True, Rank: 1.000
Column: 15, Selected False, Rank: 17.000
Column: 16, Selected False, Rank: 18.000
Column: 17, Selected False, Rank: 16.000
Column: 18, Selected False, Rank: 14.000
Column: 19, Selected False, Rank: 12.000
Column: 20, Selected False, Rank: 10.000
Column: 21, Selected False, Rank: 8.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected True, Rank: 1.000
Column: 24, Sele

In [27]:
# define RFE
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select = 10)
# fit RFE
rfe.fit(X, y)

# summarize all features
model_L         =    list()
sno_L           =    list()
support_text    =    list()
rank_list       =    list()
each            =    'Random Forest'
print("Proecessing %s" % each)

for i in range(X.shape[1]):
    model_L.append(each)
    sno_L.append(i)
    support_text.append(rfe.support_[i])
    rank_list.append(rfe.ranking_[i])
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    
df_imp_ind2   =    pd.DataFrame({'Sno' : sno_L, 'Model' : model_L, 'Selected?' : support_text, 'Rank' : rank_list, 'Columns' : collist })


Proecessing Random Forest
Column: 0, Selected False, Rank: 15.000
Column: 1, Selected False, Rank: 3.000
Column: 2, Selected False, Rank: 2.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected False, Rank: 11.000
Column: 7, Selected False, Rank: 9.000
Column: 8, Selected False, Rank: 14.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 13.000
Column: 11, Selected False, Rank: 10.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 6.000
Column: 14, Selected True, Rank: 1.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected False, Rank: 16.000
Column: 17, Selected False, Rank: 8.000
Column: 18, Selected False, Rank: 12.000
Column: 19, Selected False, Rank: 4.000
Column: 20, Selected False, Rank: 18.000
Column: 21, Selected False, Rank: 17.000
Column: 22, Selected False, Rank: 21.000
Column: 23, Selected True, Rank: 1.000
Column: 24, Selected F

In [28]:
# define RFE
rfe = RFE(estimator = LinearDiscriminantAnalysis(solver = 'svd'), n_features_to_select = 10)
# fit RFE
rfe.fit(X, y)

# summarize all features
model_L         =    list()
sno_L           =    list()
support_text    =    list()
rank_list       =    list()
each            =    'LDA'
print("Proecessing %s" % each)

for i in range(X.shape[1]):
    model_L.append(each)
    sno_L.append(i)
    support_text.append(rfe.support_[i])
    rank_list.append(rfe.ranking_[i])
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    
df_imp_ind3   =    pd.DataFrame({'Sno' : sno_L, 'Model' : model_L, 'Selected?' : support_text, 'Rank' : rank_list, 'Columns' : collist })


Proecessing LDA
Column: 0, Selected True, Rank: 1.000
Column: 1, Selected False, Rank: 10.000
Column: 2, Selected False, Rank: 14.000
Column: 3, Selected False, Rank: 12.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected False, Rank: 3.000
Column: 7, Selected False, Rank: 4.000
Column: 8, Selected False, Rank: 6.000
Column: 9, Selected False, Rank: 8.000
Column: 10, Selected False, Rank: 20.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 7.000
Column: 14, Selected False, Rank: 15.000
Column: 15, Selected False, Rank: 18.000
Column: 16, Selected True, Rank: 1.000
Column: 17, Selected False, Rank: 13.000
Column: 18, Selected False, Rank: 2.000
Column: 19, Selected False, Rank: 5.000
Column: 20, Selected False, Rank: 19.000
Column: 21, Selected True, Rank: 1.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 17.000
Column: 24, Selected False, Rank:

In [29]:
# Appending multiple DataFrame
df_imp       =    pd.DataFrame()
df_imp       =    pd.concat([df_imp_ind1, df_imp_ind2, df_imp_ind3], ignore_index=True)

In [30]:
df_imp.head().T

,0,1,2,3,4
Sno,0,1,2,3,4
Model,Decision Tree,Decision Tree,Decision Tree,Decision Tree,Decision Tree
Selected?,False,False,False,True,False
Rank,21,20,19,1,9
Columns,Year Encoded,Asset_turnover,Receivable_turnover(new),Inventory_turnover,Cash_ratio


In [31]:
df_important =    df_imp.loc[df_imp['Selected?'] == True]

In [32]:
df_important.shape

(30, 5)

In [33]:
df_important

,Sno,Model,Selected?,Rank,Columns
3,3,Decision Tree,True,1,Inventory_turnover
14,14,Decision Tree,True,1,Interest_coverage
22,22,Decision Tree,True,1,Operating Cash Flow/Total Sales
23,23,Decision Tree,True,1,Operating Cash Flow/Total Debt
24,24,Decision Tree,True,1,Operating Cash Flow/Shareholder's Equity
25,25,Decision Tree,True,1,Fixed Asset Turnover Ratio
26,26,Decision Tree,True,1,YOY Sales Growth Rate
27,27,Decision Tree,True,1,YOY EBIT Growth Rate
28,28,Decision Tree,True,1,Total shareholders' funds
29,29,Decision Tree,True,1,Shareholderquity_code


In [34]:
df_important.to_csv('./Output/Impoartant Features.csv', index = False)

In [35]:
imp_cols_list =   df_important['Columns'].tolist()

In [36]:
ul = np.unique(imp_cols_list).tolist()

In [37]:
print(ul); print(len(ul))

['Asset_coverage', 'Cash_ratio', 'EBIT_Sales', 'Fixed Asset Turnover Ratio', 'Interest_coverage', 'Inventory_turnover', 'Ln_GVA', "Operating Cash Flow/Shareholder's Equity", 'Operating Cash Flow/Total Debt', 'Operating Cash Flow/Total Sales', 'Quick_ratio', 'ROS(new)', 'Shareholderquity_code', "Total shareholders' funds", 'YOY EBIT Growth Rate', 'YOY Sales Growth Rate', 'Year Encoded', 'debt_asset', 'debt_equity']
19


### Observations

We applied three models to select important features 

Twenty columns are idemtified as important features in predicting the target variable (Bankruptcy).

They are as follows:

| S No | Column            | Remarks                                          |
|:--- | --------------------------- | ----------------------------------- |
| 1 | Asset_coverage | Selected by Random Forest model |
| 2 | Asset_turnover | Selected by Random Forest model |
| 3 | Cash_ratio | Selected by Random Forest & LDA  models |
| 4 | EBIT_Sales | Selected by LDA model |
| 5 | Fixed Asset Turnover Ratio  | Selected by Decision Tree model |
| 6 | Interest_coverage | Selected by Decision Tree & Random Forest models |
| 7 | Inventory_turnover | Selected by Decision Tree & Random Forest models |
| 8 | Ln_GVA | Selected by LDA model |
| 9 | Operating Cash Flow/Shareholder's Equity | Selected by Decision Tree model |
| 10 | Operating Cash Flow/Total Debt | Selected by Decision Tree & Random Forest models |
| 11 | Operating Cash Flow/Total Sales |  Selected by Decision Tree & LDA models  |
| 12 | Quick_ratio | Selected by LDA model |
| 13 | ROS(new) | Selected by Random Forest model |
| 14 | Shareholderquity_code | Selected by Decision Tree, Random Forest & LDA models |
| 15 | Total shareholders' funds |  Selected by Decision Tree & Random Forest |
| 16 | YOY EBIT Growth Rate | Selected by Decision Tree & LDA models  |
| 17 | YOY Sales Growth Rate |  Selected by Decision Tree model |
| 18 | Year Encoded |  Selected by LDA model |
| 19 | debt_asset | Selected by LDA model |
| 20 | debt_equity | Selected by Random Forest & LDA  models |

In [38]:
df_imputed.to_csv('./Output/imputed_dataset_all.csv', index = False)

In [39]:
imp_cols_list.insert(0, 'Target')
df_imporant_20    =  df_imputed[imp_cols_list]

In [40]:
df_imporant_20.head()

,Target,Inventory_turnover,Interest_coverage,Operating Cash Flow/Total Sales,Operating Cash Flow/Total Debt,Operating Cash Flow/Shareholder's Equity,Fixed Asset Turnover Ratio,YOY Sales Growth Rate,YOY EBIT Growth Rate,Total shareholders' funds,...,Year Encoded,Cash_ratio,Quick_ratio,debt_asset,debt_equity,EBIT_Sales,Ln_GVA,Operating Cash Flow/Total Sales,YOY EBIT Growth Rate,Shareholderquity_code
0,1,0.00000,0.00000,817.97404,29.35541,0.41067,0.00000,9627.92834,-0.30656,1439044420.00000,...,-8,2.01764,1.55202,0.01114,0.01399,817.98307,14.12832,817.97404,-0.30656,0.00000
1,1,0.00000,-9601.26389,0.41343,24.91447,0.41436,2.77352,0.06378,-0.00279,1012901796.00000,...,-7,1.55843,1.08407,0.01256,0.01663,0.38857,14.15909,0.41343,-0.00279,0.00000
2,1,0.00000,-3777.95879,0.39055,28.85392,0.55875,2.90813,0.07572,-0.02974,754886314.00000,...,-6,0.85337,0.38471,0.01449,0.01936,0.36426,14.21219,0.39055,-0.02974,0.00000
3,1,0.00000,2887.30493,0.35407,25.61557,0.53394,3.13408,0.24371,0.14724,770367465.00000,...,-5,0.96842,0.37537,0.01640,0.02084,0.32854,14.26065,0.35407,0.14724,0.00000
4,1,0.00000,11939.56046,0.32075,16.92720,0.58600,3.99127,-0.08577,-0.22265,790918031.00000,...,-4,1.21196,0.61504,0.02708,0.03462,0.30306,14.33665,0.32075,-0.22265,0.00000


In [41]:
df_imporant_20.to_csv('./Output/Important_20_columns_Data.csv', index = False)

By majority votes, the following nine columns are identified as important by at least two models

* 1) Cash_ratio
* 2) Interest_coverage
* 3) Inventory_turnover 
* 4) Operating Cash Flow/Total Debt 
* 5) Operating Cash Flow/Total Sales
* 6) Shareholderquity_code
* 7) Total shareholders' funds 
* 8) YOY EBIT Growth Rate 
* 9) debt_equity

In [42]:
consensus_columns =  ["Cash_ratio", "Interest_coverage", "Inventory_turnover", "Operating Cash Flow/Total Debt",\
"Operating Cash Flow/Total Sales", "Shareholderquity_code", "Total shareholders' funds", "YOY EBIT Growth Rate",\
"debt_equity"]

In [43]:
consensus_columns.insert(0, 'Target')
df_imporant_9    =  df_imputed[consensus_columns]
df_imporant_9.to_csv('./Output/Important_9_columns_Data.csv', index = False)                               